In [1]:
import gurobipy as gp
from gurobipy import GRB
import random
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
import numpy as np
import warnings
from tqdm import tqdm_notebook as tqdm
import requests
import json
import API
import matplotlib.pyplot as plt
import GoogleDistanceMatrix
import GoogleVRP
import routing
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)
warnings.filterwarnings('ignore')

## modelling

In [137]:
order = pd.read_csv('Data/ProcessedNov.csv')

In [138]:
order['門店名稱'].unique()

array(['PH64', 'PH57', 'PHD629', 'PHD606'], dtype=object)

In [139]:
phdstore = 'PHD606'
phstore = 'PH64'

In [140]:
order = order.loc[order['門店名稱'].isin([phdstore,phstore])]
order.shape

(193, 30)

In [141]:
order['foodPreparationTime'] = pd.to_datetime(order['取餐出發時間']) - pd.to_datetime(order['下單時間'])

In [142]:
order = order[['orderTime', 'arrivalTime', 'foodPreparationTime','waitingTime', 'departureTime', 'tripDurationReal',
       'DeliveryLatLon', 'storeCode', 'StoreLatLon',
       'tripDurationFromGoogle', 'shared']]

In [143]:
slot = 1
order['departureTime'] = pd.to_datetime(order['departureTime'])
order['hour'] = order['departureTime'].dt.hour
order['min'] = order['departureTime'].dt.minute
order['min'] = order['min'].apply(lambda x: (int(x/slot)+1)*slot)

In [144]:
order['timeWindow'] = pd.Timedelta('0 days 00:45:00') - order['foodPreparationTime']
order['timeWindow'] = order['timeWindow'].dt.seconds


In [145]:
order['timeParameter'] = order['tripDurationReal']/order['tripDurationFromGoogle']
order['timeParameter'] = order['timeParameter'].median()

In [146]:
order[order['tripDurationReal'] < 60]['tripDurationReal'] = order['timeParameter'].median() * order[order['tripDurationReal'] < 60]['tripDurationFromGoogle']

In [147]:
order['date'] = pd.to_datetime(order['orderTime']).dt.date

In [148]:
# dispatch immediately
order['timeWindow'] = order['timeWindow'] /order['timeParameter']
order['timeWindow'] = order['tripDurationFromGoogle']+10

In [122]:
# slot aggregation
order['timeWindow'] = order['timeWindow'] /order['timeParameter']
order.loc[order['timeWindow'] < order['tripDurationFromGoogle'],'timeWindow'] = \
order.loc[order['timeWindow'] < order['tripDurationFromGoogle'],'tripDurationFromGoogle']+10

In [149]:
rider_df = pd.read_excel('../1. Sales Data/2-6Nov - Clock in record.xls')
rider_df['Clock in.1'] = pd.to_datetime(rider_df['Clock in.1'])
rider_df['Date'] = rider_df['Clock in.1'].dt.day

In [150]:
PHDLatLon = order.loc[order['storeCode']==phdstore]['StoreLatLon'].values[0]
PHLatLon = order.loc[order['storeCode']==phstore]['StoreLatLon'].values[0]


In [151]:
order['tripTime'] = 0
order['rider'] = 0

In [152]:
PHD_output = pd.DataFrame()
PH_output = pd.DataFrame()

In [153]:
order['tripTime'] = 0
order['rider'] = 0
order['delayed'] = 0
order['departureTimeSimulation'] = pd.to_datetime(order.orderTime).dt.date.astype(str)+' '+order['hour'].apply(lambda x:str(x).zfill(2))+\
                                        ':'+order['min'].apply(lambda x:str(x).zfill(2))+':00'
PHD_output = pd.DataFrame()
PH_output = pd.DataFrame()

In [164]:
order.shape

(193, 20)

In [154]:
for day in range(2,7):
    # reset riders
    riders = rider_df.loc[(rider_df['Rider Store Code'].isin([int(phstore.replace('PH','')),int(phdstore.replace('PHD',''))]))&(rider_df['Date']==day)]['Rider No.'].unique()
    riderStatus = {}
    for i in (riders):
        riderStatus[i] = 0
    for hour in range(11,24):
        for minute in np.arange(0,60,slot):
            print(day,hour,minute)
            # update rider status at current time
            for key in riderStatus.keys():
                if riderStatus[key]!= 0:
                    if riderStatus[key] <= pd.to_datetime('2020-11-0%s %s:%s:00'%(day,hour,str(minute).zfill(2))):
                        riderStatus[key] = 0

            candidate = order.loc[(pd.to_datetime(order['orderTime']).dt.day==day)&(order['hour']==hour)&(order['min']==minute)]
            if len(candidate) > 0:
                PHD = candidate.loc[candidate['storeCode']==phdstore]
                PH = candidate.loc[candidate['storeCode']==phstore]
                timeParameter = order.loc[(order['hour']==hour)&(order['min']==minute)]['timeParameter'].unique()[0]
                print('PHD')
                PHD_output,riderStatus,order = routing.tripAssign(day,hour,minute,timeParameter,PHD, PHDLatLon, riderStatus,order,PHD_output)
                print('PH')
                PH_output,riderStatus,order = routing.tripAssign(day,hour,minute,timeParameter,PH, PHLatLon, riderStatus,order,PH_output)

                

2 11 0
2 11 1
2 11 2
2 11 3
2 11 4
2 11 5
2 11 6
2 11 7
2 11 8
2 11 9
2 11 10
2 11 11
2 11 12
2 11 13
2 11 14
2 11 15
2 11 16
2 11 17
2 11 18
2 11 19
2 11 20
2 11 21
2 11 22
2 11 23
2 11 24
2 11 25
2 11 26
2 11 27
2 11 28
2 11 29
2 11 30
2 11 31
2 11 32
2 11 33
2 11 34
2 11 35
PHD
PH
2 11 36
2 11 37
2 11 38
2 11 39
2 11 40
2 11 41
2 11 42
2 11 43
2 11 44
2 11 45
2 11 46
2 11 47
2 11 48
2 11 49
2 11 50
2 11 51
2 11 52
2 11 53
2 11 54
2 11 55
2 11 56
2 11 57
2 11 58
2 11 59
2 12 0
2 12 1
2 12 2
2 12 3
2 12 4
2 12 5
2 12 6
2 12 7
2 12 8
2 12 9
2 12 10
2 12 11
2 12 12
2 12 13
2 12 14
2 12 15
2 12 16
2 12 17
2 12 18
2 12 19
2 12 20
2 12 21
2 12 22
2 12 23
2 12 24
2 12 25
2 12 26
2 12 27
2 12 28
2 12 29
2 12 30
2 12 31
2 12 32
2 12 33
PHD
PH
2 12 34
2 12 35
2 12 36
2 12 37
2 12 38
2 12 39
2 12 40
2 12 41
2 12 42
2 12 43
2 12 44
2 12 45
PHD
PH
2 12 46
2 12 47
2 12 48
2 12 49
2 12 50
2 12 51
2 12 52
2 12 53
2 12 54
2 12 55
2 12 56
2 12 57
2 12 58
2 12 59
2 13 0
2 13 1
2 13 2
2 13 3
2 13 4
2 13

3 16 48
3 16 49
3 16 50
3 16 51
3 16 52
3 16 53
3 16 54
3 16 55
3 16 56
PHD
PH
3 16 57
3 16 58
3 16 59
3 17 0
3 17 1
3 17 2
3 17 3
3 17 4
3 17 5
3 17 6
3 17 7
3 17 8
3 17 9
3 17 10
3 17 11
3 17 12
3 17 13
3 17 14
3 17 15
PHD
PH
3 17 16
3 17 17
3 17 18
3 17 19
3 17 20
3 17 21
3 17 22
3 17 23
3 17 24
3 17 25
3 17 26
3 17 27
3 17 28
3 17 29
PHD
PH
3 17 30
3 17 31
3 17 32
3 17 33
3 17 34
3 17 35
3 17 36
3 17 37
3 17 38
3 17 39
3 17 40
3 17 41
3 17 42
3 17 43
3 17 44
3 17 45
3 17 46
3 17 47
3 17 48
3 17 49
3 17 50
3 17 51
3 17 52
3 17 53
3 17 54
PHD
PH
3 17 55
3 17 56
3 17 57
3 17 58
3 17 59
3 18 0
3 18 1
3 18 2
3 18 3
3 18 4
3 18 5
3 18 6
3 18 7
PHD
PH
3 18 8
3 18 9
3 18 10
3 18 11
3 18 12
3 18 13
3 18 14
3 18 15
3 18 16
3 18 17
3 18 18
3 18 19
3 18 20
3 18 21
3 18 22
PHD
PH
3 18 23
3 18 24
3 18 25
PHD
PH
3 18 26
3 18 27
3 18 28
3 18 29
3 18 30
3 18 31
3 18 32
3 18 33
3 18 34
3 18 35
3 18 36
3 18 37
3 18 38
3 18 39
3 18 40
3 18 41
3 18 42
PHD
PH
3 18 43
3 18 44
PHD
PH
3 18 45
3 18 46
3 18 

PH
4 20 48
4 20 49
4 20 50
4 20 51
4 20 52
4 20 53
4 20 54
4 20 55
4 20 56
4 20 57
PHD
PH
4 20 58
4 20 59
4 21 0
4 21 1
4 21 2
4 21 3
4 21 4
4 21 5
4 21 6
4 21 7
4 21 8
4 21 9
4 21 10
4 21 11
4 21 12
4 21 13
4 21 14
4 21 15
4 21 16
4 21 17
4 21 18
4 21 19
4 21 20
4 21 21
4 21 22
4 21 23
4 21 24
4 21 25
4 21 26
4 21 27
4 21 28
4 21 29
4 21 30
4 21 31
4 21 32
4 21 33
4 21 34
PHD
PH
4 21 35
4 21 36
4 21 37
4 21 38
4 21 39
4 21 40
4 21 41
4 21 42
4 21 43
4 21 44
4 21 45
4 21 46
4 21 47
4 21 48
4 21 49
4 21 50
4 21 51
4 21 52
4 21 53
4 21 54
4 21 55
4 21 56
4 21 57
4 21 58
4 21 59
4 22 0
4 22 1
4 22 2
4 22 3
4 22 4
4 22 5
4 22 6
4 22 7
4 22 8
4 22 9
4 22 10
4 22 11
4 22 12
4 22 13
4 22 14
4 22 15
4 22 16
4 22 17
4 22 18
4 22 19
4 22 20
4 22 21
4 22 22
4 22 23
4 22 24
4 22 25
4 22 26
4 22 27
4 22 28
4 22 29
4 22 30
4 22 31
4 22 32
4 22 33
4 22 34
4 22 35
4 22 36
4 22 37
4 22 38
4 22 39
4 22 40
4 22 41
4 22 42
4 22 43
4 22 44
4 22 45
4 22 46
4 22 47
4 22 48
4 22 49
4 22 50
4 22 51
4 22 52
4 2

PH
6 11 54
6 11 55
6 11 56
6 11 57
6 11 58
6 11 59
6 12 0
6 12 1
6 12 2
6 12 3
6 12 4
6 12 5
PHD
PH
6 12 6
6 12 7
6 12 8
6 12 9
6 12 10
6 12 11
6 12 12
6 12 13
6 12 14
6 12 15
6 12 16
6 12 17
6 12 18
6 12 19
6 12 20
6 12 21
6 12 22
6 12 23
6 12 24
6 12 25
PHD
PH
6 12 26
6 12 27
6 12 28
6 12 29
6 12 30
6 12 31
6 12 32
6 12 33
PHD
PH
6 12 34
6 12 35
6 12 36
6 12 37
6 12 38
6 12 39
6 12 40
6 12 41
6 12 42
6 12 43
6 12 44
6 12 45
6 12 46
6 12 47
6 12 48
6 12 49
6 12 50
6 12 51
6 12 52
PHD
PH
6 12 53
6 12 54
PHD
PH
6 12 55
6 12 56
6 12 57
PHD
PH
6 12 58
6 12 59
6 13 0
6 13 1
6 13 2
6 13 3
6 13 4
PHD
PH
6 13 5
6 13 6
6 13 7
6 13 8
6 13 9
6 13 10
6 13 11
6 13 12
6 13 13
6 13 14
6 13 15
6 13 16
6 13 17
PHD
PH
6 13 18
6 13 19
6 13 20
6 13 21
6 13 22
6 13 23
6 13 24
PHD
PH
6 13 25
6 13 26
6 13 27
6 13 28
6 13 29
6 13 30
6 13 31
6 13 32
6 13 33
6 13 34
6 13 35
6 13 36
6 13 37
6 13 38
6 13 39
6 13 40
6 13 41
6 13 42
6 13 43
6 13 44
6 13 45
6 13 46
6 13 47
6 13 48
6 13 49
6 13 50
6 13 51
6 13 52
6 

In [155]:
output = pd.concat([PHD_output,PH_output])

In [165]:
output.shape

(190, 23)

In [156]:
output['arrivalTimeSimulation'] = pd.to_datetime(output['departureTimeSimulation']) +\
                                        pd.to_timedelta(output['tripTime'],'seconds')
output['waitingTimeSimulation'] = output['arrivalTimeSimulation'] - pd.to_datetime(output['orderTime'])
output['waitingTimeReal'] = pd.to_datetime(output['arrivalTime']) - pd.to_datetime(output['orderTime'])


In [157]:
output['waitingTimeSimulation'].mean()

Timedelta('0 days 00:32:23.675761')

In [158]:
output['waitingTimeReal'].mean()

Timedelta('0 days 00:32:46.426315')

In [159]:
output.shape

(190, 23)

In [163]:
output.to_csv('Output/order64_606_immediately_minTW.csv',index=False)

In [162]:
output.loc[output['rider']=='3rd']

,orderTime,arrivalTime,foodPreparationTime,waitingTime,departureTime,tripDurationReal,DeliveryLatLon,storeCode,StoreLatLon,tripDurationFromGoogle,shared,hour,min,timeWindow,timeParameter,date,tripTime,rider,delayed,departureTimeSimulation,arrivalTimeSimulation,waitingTimeSimulation,waitingTimeReal
40,2020-11-02 18:12:00,2020-11-02 18:34:27,00:16:11,0 days 00:22:27.000000000,2020-11-02 18:28:11,376,"22.2843523,114.145566",PHD606,"22.284361999999998,114.13979259999999",169,0,18,29,179,1.986842,2020-11-02,335.776316,3rd,0,2020-11-02 18:29:00,2020-11-02 18:34:35.776315789,00:22:35.776315,00:22:27
311,2020-11-05 18:16:00,2020-11-05 18:36:50,00:11:03,0 days 00:20:50.000000000,2020-11-05 18:27:03,587,"22.2816741,114.1247985",PHD606,"22.284361999999998,114.13979259999999",513,0,18,28,523,1.986842,2020-11-05,1021.236842,3rd,0,2020-11-05 18:28:00,2020-11-05 18:45:01.236842105,00:29:01.236842,00:20:50
42,2020-11-02 18:17:00,2020-11-02 18:47:57,00:15:06,0 days 00:30:57.000000000,2020-11-02 18:32:06,951,"22.283516,114.129613",PH64,"22.284392699999998,114.141973",565,0,18,33,575,1.986842,2020-11-02,951.000000,3rd,0,2020-11-02 18:32:06,2020-11-02 18:47:57.000000000,00:30:57,00:30:57
